# Mapping

Using the [DwC-A_dotnet.Mapping](https://www.nuget.org/packages/DwC-A_dotnet.Mapping/) library we can map data from a [DwC-A_dotnet](https://www.nuget.org/packages/DwC-A_dotnet/) IRow to a strongly typed class.  There are two different approaches to mapping data

1. Using the dwca-codegen magic command to generate a class and mapping method from archive metadata.  This requires the least code and is more interactive but can run into more issues.
1. Manually create a class definition and mapping method.  This gives the most control over mapping but requires more effort. 

In [1]:
#r "nuget:DwC-A_dotnet.Interactive,0.1.12-Pre"
#r "nuget:DwC-A_dotnet.Mapping,0.6.3"
#r "nuget:Microsoft.ML"

Installed Packages DwC-A_dotnet.Interactive, 0.1.12-pre DwC-A_dotnet.Mapping, 0.6.3 Microsoft.ML, 1.7.1

Loading extensions from `DwC-A_dotnet.Interactive.dll`

## Using The dwca-codegen Magic Command

First, we will map using the dwca-codegen magic command.

### Create Configuration

This step is optional but gives more control over how classes are generated and mapped.  If this step is left out all properties will be mapped as string type.

Use the GeneratorConfigurationBuilder to create a configuration to influence how the dwca-codegen command generates the class to be mapped for each file in the archive.

Use the AppProperty method to define the properties that will be added to the generated classes and how they will be mapped to specific terms.  Use the wildcard term __*__ to determine whether properties will be created for all other undefined terms and mapped.

The WithMapMethod creates a static method on the type called MapRow that will be used to map an IRow row to an instance of the generated class.

In [2]:
using DwC_A.Interactive.Mapping;
using DwC_A.Terms;

var config = new GeneratorConfigurationBuilder()
.AddProperty("*", "string", true)
.AddProperty(Terms.decimalLatitude, "double", true, "Latitude")
.AddProperty(Terms.decimalLongitude, "double", true, "Longitude")
.AddProperty(Terms.dateIdentified, "DateTime")
.WithMapMethod(true)
.Build();

config

Options Option Value Namespace MapMethod True Output PascalCase True TermAttribute none Usings System DwC_A DwC_A.Extensions Properties Name Type Include Term <null> string True * Latitude double True http://rs.tdwg.org/dwc/terms/decimalLatitude Longitude double True http://rs.tdwg.org/dwc/terms/decimalLongitude <null> DateTime True http://rs.tdwg.org/dwc/terms/dateIdentified

### dwca-codegen

Use the dwca-codegen command to examine the archive and generate classes to map data into.  The --configName option can be used to specify the name of the variable that contains the configuration information we created earlier. 

In [3]:
#!dwca-codegen -h
#!dwca-codegen -c config ./data/Papilionidae.zip

Description:
  Generate strongly typed class files for Darwin Core Archive

Usage:
  #!dwca-codegen <archivePath> [options]

Arguments:
  <archivePath>  Path to archive folder or zip file

Options:
  -c, --configName <configName>  Name of configuration variable []
  -?, -h, --help                 Show help and usage information





Opening archive ./data/Papilionidae.zip using configuration

Options Option Value Namespace MapMethod True Output PascalCase True TermAttribute none Usings System DwC_A DwC_A.Extensions Properties Name Type Include Term <null> string True * Latitude double True http://rs.tdwg.org/dwc/terms/decimalLatitude Longitude double True http://rs.tdwg.org/dwc/terms/decimalLongitude <null> DateTime True http://rs.tdwg.org/dwc/terms/dateIdentified

Generating class Occurrence

Generating class Multimedia

Generating class Verbatim

## Create Mapper

Now that we have class definitions and a mapping method we can define a mapper as follows.

In [4]:
using DwC_A.Mapping;

var mapper = MapperFactory.CreateMapper<Occurrence>(Occurrence.MapRow);


## Map Archive

Finally, we can open the archive and query rows that we can map using the mapper.

**Hint:** Use the Greedy RowStrategy for better performance when mapping the entire class.

There are three different Map extensions for the IFileReader and IRow interfaces returned by the ArchiveReader.

In [5]:
using DwC_A;
using DwC_A.Factories;
using DwC_A.Config;

var factory = new DefaultFactory((cfg) => {
    cfg.Add<RowFactoryConfiguration>(c => c.Strategy = RowStrategy.Greedy);
});

var archive = new ArchiveReader("./data/Papilionidae.zip", factory);

var occurrences = archive.CoreFile
    .DataRows
    .Where(row => row[Terms.decimalLatitude] != null)
    .Where(row => row[Terms.dateIdentified] != null)
    .Map<Occurrence>(mapper);

occurrences.Select(o => new {
    o.ScientificName,
    o.Latitude,
    o.Longitude,
    o.DateIdentified
})

index,ScientificName,Latitude,Longitude,DateIdentified
0,"Battus philenor (Linnaues, 1771)",32.996571,-97.148685,2021-02-23 02:11:11Z
1,"Eurytides marcellus (Cramer, 1777)",32.754545,-94.483826,2020-09-25 21:31:00Z
2,"Battus philenor (Linnaues, 1771)",29.824338,-104.307482,2021-02-19 04:44:44Z
3,"Battus philenor (Linnaues, 1771)",31.590366,-98.927223,2021-02-23 18:21:20Z
4,"Papilio polibetes Stoll, 1781",33.737175,-96.576508,2021-02-21 03:35:16Z
5,"Papilio glaucus Linnaeus, 1758",30.693989,-97.822322,2021-02-25 02:22:19Z
6,"Papilio palamedes Drury, 1773",28.244524,-96.856559,2021-02-25 03:41:59Z
7,"Battus philenor (Linnaues, 1771)",29.386342,-95.011422,2021-02-25 18:42:07Z
8,"Papilio cresphontes Cramer, 1777",30.924697,-94.003875,2020-08-25 21:01:29Z
9,"Papilio multicaudata Kirby, 1884",29.552946,-98.233367,2021-02-12 15:13:22Z


## Manual Mapping

If you already have a class definition or want to create the class definition by hand then use this method.

Classes may be defined two ways.

1. Defined directly in a cell.
1. Loaded from a file on disk using the #load magic command.

In this instance we'll load the class definition from a file.  After that we can define a mapper and map method.

In [6]:
#load "./Code/Multimedia.cs"

using System;

var multimediaMapper = MapperFactory.CreateMapper<Multimedia>((m, row) => {
    m.GbifID = row.Convert<long>("http://rs.gbif.org/terms/1.0/gbifID");
    m.Type = row["http://purl.org/dc/terms/type"];
    m.Identifier = row["http://purl.org/dc/terms/identifier"];
    m.Created = row.Convert<DateTime>("http://purl.org/dc/terms/created");
});

archive.Extensions
    .GetFileReadersByRowType("http://rs.gbif.org/terms/1.0/Multimedia")
    .FirstOrDefault()?
    .Map<Multimedia>(multimediaMapper)

index,GbifID,Type,Identifier,Created,Creator
0,3044911996,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113743750/original.jpg?1614044752,2020-09-14 12:35:15Z,<null>
1,3044896703,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113394978/original.jpg?1613765341,2020-04-06 19:36:03Z,<null>
2,3044892607,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113344109/original.jpeg?1613707791,2006-04-30 11:47:01Z,<null>
3,3044876974,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113796898/original.jpg?1614103220,2003-06-25 11:55:00Z,<null>
4,3044875694,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113535495/original.jpg?1613878613,2020-10-09 06:45:42Z,<null>
5,3044875210,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113937636/original.jpg?1614218182,2015-05-09 12:53:30Z,<null>
6,3044864241,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113945923/original.jpg?1614224485,2016-04-09 12:37:20Z,<null>
7,3044857233,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113993610/original.jpg?1614278545,2021-02-25 08:55:07Z,<null>
8,3044857233,StillImage,https://inaturalist-open-data.s3.amazonaws.com/photos/113993597/original.jpg?1614278538,2021-02-25 08:55:07Z,<null>
9,3044837653,StillImage,https://static.inaturalist.org/photos/91859858/original.jpg?1598389359,2020-08-13 07:22:56Z,<null>


## Using With Microsoft.ML

Now that we have mapped an IEnumerable of Occurrences we can load the data into an IDataView or DataFrame using MLContext from [Microsoft.ML](https://www.nuget.org/packages/Microsoft.ML/).

In [7]:
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext();

var data = mlContext.Data.LoadFromEnumerable<Occurrence>(occurrences);

data.Schema

index Name Index IsHidden Type Annotations 0 GbifID 0 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 1 Abstract 1 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 2 AccessRights 2 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 3 AccrualMethod 3 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 4 AccrualPeriodicity 4 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 5 AccrualPolicy 5 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 6 Alternative 6 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 7 Audience 7 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 8 Available 8 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 9 BibliographicCitation 9 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 10 ConformsTo 10 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 11 Contributor 11 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 12 Coverage 12 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 13 Created 13 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 14 Creator 14 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 15 Date 15 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 16 DateAccepted 16 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 17 DateCopyrighted 17 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 18 DateSubmitted 18 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 19 Description 19 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] ... (more)